In [ ]:
#
# Valuation European Call Options in BSM Model
# Comparison of Analytical, int_valueegral and FFT (Fast Fourier Transform) Approach
#
import math
import numpy as np
from numpy.fft import fft
from scipy.integrate import quad
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

#
# Model Parameters
#
S0 = 100.00 # initial index level
K = 100.00 # strike level
T = 1. # call option maturity
r = 0.05 # constant short rate
sigma = 0.2 # constant volatility of diffusion

#
# Vvaluation y int_valueegration
#
### Analytical Formula
def BSM_call_value(S0, K, T, r, sigma):
    ''' Valuation of European call option in BSM Model.
    --> Analytical Formula.
    
    Parameters
    =============
    S0: float
        initial stock/index level
    K:  float
        strike price
    T:  float
        time-to-maturity (for t=0)
    r:  float
        constant risk-free short rate
    sigma: float
           volatility factor in diffusion term
           
    Returns
    =========
    call_value: float
                European call option present value
                
    '''
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    BS_C = (S0 * stats.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * stats.norm.cdf(d2, 0.0, 1.0))

    return BS_C


#
# Fourier Transform with Numerical int_valueegration
#

def BSM_call_value_INT(S0, K, T, r, sigma):
    ''' Valuation of European call option in BSM model via Lewis (2001)
    --> Fourier-based approach (integral).
    
    Parameters
    ============
    S0: float
        initial stock/index level
    K: float
       strike price
    T: float
       time-to-maturity (for t=0)
    r: float
       constant risk-free short rate
    sigma: float
           volatility factor in diffusion term
    
    Returns
    =========
    call_value: float
                European call option present value
    '''
    int_value = quad(lambda u: BSM_integral_function(u, S0, K, T, r, sigma), 0, 100)[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
    return call_value

def BSM_integral_function(u, S0, K, T, r, sigma):
    ''' Valuation of European call option in BSM model via Lewis (2001)
    --> Fourier-based approach: integral function. '''
    cf_value = BSM_characteristic_function(u -1j * 0.5, 0.0, T, r, sigma)
    int_value  = 1 / (u ** 2 + 0.25) * (np.exp(1j * u * np.log(S0 / K)) * cf_value).real
    return int_value

def BSM_characteristic_function(v, x0, T, r, sigma):
    ''' Valuation of European call option in BSM model via Lewis (2001) and Carr-Madan (1999)
    --> Fourier-based approach: characteristic function. '''
    cf_value = np.exp(((x0 / T + r - 0.5 * sigma ** 2) * 1j * v - 0.5 * sigma ** 2 * v ** 2) * T)
    return cf_value
    